# DSA 1080A/VA — Exam Prep Notebook (FS 2025)

**Sections covered:** From **Working With CSV Files** through **Debugging & Code Correction**.
This notebook contains content‑intensive practice questions and runnable code for each item.

> Tip: Replace file names like `sales.csv` with your actual paths.


In [3]:

# --- Setup & environment ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)


## Working With CSV Files
### 1) Reading data into a DataFrame using CSV — Practice Questions
- Q1. Load a local CSV called `sales.csv` into a DataFrame and parse `date` as datetime.- Q2. Read a `;`-delimited file with `ISO-8859-1` encoding; coerce `revenue` to numeric.- Q3. Read a very large CSV in chunks of 50,000 rows and compute total revenue.- Q4. Compute a new `revenue = price * quantity` after reading.


In [4]:

# Q1: Basic read with date parsing
# Replace 'sales.csv' with your actual file path.
df = pd.read_csv(r'C:\Users\T2Gic\Downloads\sales_q1.csv', parse_dates=['date'])
df.head()


,date,region,product,category,units,unit_price
0,2025-01-02,Nairobi,Chai,Drinks,120,2.5
1,2025-01-05,Mombasa,Coffee,Drinks,80,3.0
2,2025-01-09,Nairobi,Bread,Bakery,200,1.2
3,2025-02-01,Kisumu,Chai,Drinks,95,2.5
4,2025-02-10,Naivasha,Bread,Bakery,180,1.1


In [6]:

# Q2: Custom delimiter + encoding + numeric coercion
# Replace 'sales_semicolon.csv' with your actual file path.
df = pd.read_csv(
    r'C:\Users\T2Gic\Downloads\sales_q1.csv',
    sep=';',
    encoding='ISO-8859-1',
    dtype={'category': 'string'},
    parse_dates=['date']
)
df['revenue'] = pd.to_numeric(df.get('revenue'), errors='coerce').fillna(0)
df.head()


ValueError: Missing column provided to 'parse_dates': 'date'

In [ ]:

# Q3: Chunked reading for large files
# Replace 'sales_big.csv' with your actual file path.
total_revenue = 0
for chunk in pd.read_csv('sales_big.csv', chunksize=50_000):
    chunk['revenue'] = pd.to_numeric(chunk['revenue'], errors='coerce').fillna(0)
    total_revenue += chunk['revenue'].sum()
print(f'Total revenue (chunked): {total_revenue:,.2f}')


In [ ]:

# Q4: Compute revenue while reading (vectorized after load)
# Replace 'sales_prices.csv' with your actual file path.
df = pd.read_csv('sales_prices.csv', parse_dates=['date'])
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').fillna(0)
df['revenue'] = df['price'] * df['quantity']
df.head()


### 2) Displaying sample records
- Q1. Show the top 5 rows, then the bottom 3.- Q2. Randomly sample 10 rows reproducibly.- Q3. Show 5 rows per category (stratified sampling).


In [ ]:

# Q1: Head & tail
print(df.head(5))
print(df.tail(3))


In [ ]:

# Q2: Reproducible random sample
print(df.sample(10, random_state=42))


In [ ]:

# Q3: Stratified sample: 5 rows per category (if available)
sampled = df.groupby('category', group_keys=False).apply(lambda g: g.head(5))
print(sampled)


### 3) Confirming column names and data types
- Q1. List all column names and dtypes; ensure `date` is datetime and `revenue` numeric.- Q2. Fix trailing whitespace in `'revenue '` by renaming to `'revenue'`.- Q3. Convert `category` to categorical dtype; cast `id` to nullable integer.


In [ ]:

# Q1: Inspect names & dtypes
print(df.columns.tolist())
print(df.dtypes)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')


In [ ]:

# Q2: Normalize column names; fix specific typo
df.columns = [c.strip() for c in df.columns]
df = df.rename(columns={'revenue ': 'revenue'})
df.head(3)


In [ ]:

# Q3: Cast dtypes appropriately
df['category'] = df['category'].astype('category')
df['id'] = pd.to_numeric(df['id'], errors='coerce').astype('Int64')
df.dtypes


## Charts & Visualization
### 1) Plotting with Matplotlib from DataFrames — Practice Questions
- Q1. Create a daily revenue line chart: x = `date`, y = `revenue`.- Q2. Create a grouped bar chart: total revenue per category by month (Jan/Feb/Mar).- Q3. Create a stacked bar chart of monthly revenue by category.- Q4. Create a pie chart of revenue share by category for Q1 only.


In [ ]:

# Precompute helpful columns
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['month'] = df['date'].dt.month_name().str[:3]
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce').fillna(0)


In [ ]:

# Q1: Daily revenue line chart
daily = df.groupby('date')['revenue'].sum().sort_index()
plt.figure(figsize=(10, 6))
plt.plot(daily.index, daily.values, marker='o', linestyle='-')
plt.xlabel('Date')
plt.ylabel('Total Revenue')
plt.title('Daily Revenue Trends (Q1)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

# Q2: Grouped bar chart (month × category)
pivot_grouped = df.pivot_table(index='month', columns='category', values='revenue', aggfunc='sum')
pivot_grouped = pivot_grouped.reindex(['Jan', 'Feb', 'Mar'])
ax = pivot_grouped.plot(kind='bar', figsize=(10, 6))
ax.set_xlabel('Month'); ax.set_ylabel('Total Revenue'); ax.set_title('Revenue per Category by Month (Jan–Mar)')
plt.xticks(rotation=0); plt.legend(title='Category'); plt.tight_layout(); plt.show()


In [ ]:

# Q3: Stacked bar chart (month × category)
ax = pivot_grouped.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='tab20')
ax.set_xlabel('Month'); ax.set_ylabel('Total Revenue'); ax.set_title('Monthly Revenue by Category (Stacked)')
plt.xticks(rotation=0); plt.tight_layout(); plt.show()


In [ ]:

# Q4: Pie chart (Q1 share by category)
q1 = df[df['month'].isin(['Jan', 'Feb', 'Mar'])].groupby('category')['revenue'].sum().sort_values(ascending=False)
plt.figure(figsize=(7, 7))
plt.pie(q1.values, labels=q1.index, autopct='%1.1f%%', startangle=90)
plt.title('Q1 Revenue Share by Category'); plt.tight_layout(); plt.show()


### 2) Labeling axes and adding titles
- Q1. Add axis labels & title; rotate x‑ticks for readability.- Q2. Format y‑axis as currency and add a grid.- Q3. Add a legend for multiple series.


In [ ]:

# Q1: Labels + title + rotated ticks
plt.figure(figsize=(10, 6))
plt.plot(daily.index, daily.values, 'o-', label='Total Revenue')
plt.xlabel('Date'); plt.ylabel('Revenue (KES)')
plt.title('Daily Revenue Trends')
plt.xticks(rotation=45); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:

# Q2: Currency formatting + grid
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(daily.index, daily.values, 'o-')
ax.set_xlabel('Date'); ax.set_ylabel('Revenue (KES)')
ax.set_title('Daily Revenue (Currency Format)')
ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('KES {:,.0f}'))
ax.grid(True, alpha=0.3)
fig.autofmt_xdate()
plt.tight_layout(); plt.show()


In [ ]:

# Q3: Legend with multiple series
cats = ['Electronics', 'Grocery']
multi = df[df['category'].isin(cats)].groupby(['date','category'])['revenue'].sum().unstack('category').sort_index()
fig, ax = plt.subplots(figsize=(10, 6))
for c in multi.columns:
    ax.plot(multi.index, multi[c], marker='o', label=c)
ax.set_xlabel('Date'); ax.set_ylabel('Revenue (KES)')
ax.set_title('Daily Revenue by Selected Categories')
ax.legend(title='Category'); ax.grid(True, alpha=0.3)
plt.tight_layout(); plt.show()


### 3) Understanding chart types and when to use each
- Q1. Line vs. bar for daily revenue — plot both and compare.- Q2. Grouped vs. stacked bars — Jan–Mar by category.- Q3. Pie vs. bar for category shares — discuss readability.


In [ ]:

# Q1: Line vs. bar for daily revenue
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
axes[0].plot(daily.index, daily.values, 'o-'); axes[0].set_title('Line: Daily Revenue')
axes[1].bar(daily.index, daily.values); axes[1].set_title('Bar: Daily Revenue')
for ax in axes: ax.set_xlabel('Date'); ax.set_ylabel('Revenue'); ax.tick_params(axis='x', rotation=45)
plt.tight_layout(); plt.show()


In [ ]:

# Q2: Grouped vs. stacked for Jan–Mar by category
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharex=True)
pivot_grouped.plot(kind='bar', ax=axes[0]); axes[0].set_title('Grouped Bars (Compare Categories)')
pivot_grouped.plot(kind='bar', stacked=True, ax=axes[1]); axes[1].set_title('Stacked Bars (Total per Month)')
for ax in axes: ax.set_xlabel('Month'); ax.set_ylabel('Revenue'); ax.tick_params(axis='x', rotation=0)
plt.tight_layout(); plt.show()


In [ ]:

# Q3: Pie vs. bar for category shares
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].pie(q1.values, labels=q1.index, autopct='%1.1f%%'); axes[0].set_title('Pie: Q1 Share')
axes[1].bar(q1.index, q1.values); axes[1].set_title('Bar: Q1 Share'); axes[1].tick_params(axis='x', rotation=45)
plt.tight_layout(); plt.show()


## Debugging & Code Correction (Very Important)
### 1) Identifying syntax errors in Pandas, NumPy, and Matplotlib code — Practice & Fixes
- Q1. Spot the missing comma in `plt.plot(...)`.- Q2. Find the unmatched parenthesis in `pd.to_numeric(...)`.- Q3. Replace smart quotes with ASCII quotes.


In [ ]:

# Q1: Missing comma — fixed
plt.plot(daily.index, daily.values, marker='o', linestyle='-')

# Q2: Unmatched parenthesis — fixed
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')

# Q3: Smart quotes replaced
plt.ylabel('Revenue (KES)')


### 2) Fixing incorrect imports — Practice & Fixes


In [ ]:

# Correct imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


### 3) Fixing wrong DataFrame method calls — Practice & Fixes


In [ ]:

# Correct grouping call
df.groupby(['month','category'])['revenue'].sum()

# Access column with spaces using bracket notation
_ = df['revenue total'].mean() if 'revenue total' in df.columns else None


### 4) Fixing mismatched parentheses/brackets — Practice & Fixes


In [ ]:

# Close the plotting call properly
plt.plot(daily.index, daily.values, marker='o', linestyle='-')

# List & tuple literals correct
x = [1, 2, 3]
y = (4, 5, 6)


### 5) Fixing typos in column names — Practice & Fixes


In [ ]:

# Robust rename if 'Revenue' exists
if 'Revenue' in df.columns and 'revenue' not in df.columns:
    df = df.rename(columns={'Revenue': 'revenue'})

# Normalize columns: lowercase, strip spaces, replace spaces with underscores
df.columns = (
    pd.Index(df.columns)
      .str.strip()
      .str.lower()
      .str.replace(r"\s+", "_", regex=True)
)
df.head(3)


### 6) Fixing errors in chart creation commands — Practice & Fixes


In [ ]:

# Sort by date before plotting
daily = df.groupby('date')['revenue'].sum().sort_index()
plt.plot(daily.index, daily.values, 'o-')
plt.show()

# Top-5 categories only for clarity
q1_totals = df[df['month'].isin(['Jan','Feb','Mar'])].groupby('category')['revenue'].sum().sort_values(ascending=False)
top5 = q1_totals.head(5).index
pivot_top5 = df[df['category'].isin(top5)].pivot_table(index='month', columns='category', values='revenue', aggfunc='sum')
pivot_top5 = pivot_top5.reindex(['Jan','Feb','Mar'])
pivot_top5.plot(kind='bar'); plt.show()

# Dimension mismatch fix example
x = [2, 4, 6, 8, 10]
y = [2, 3, 5, 7, 6]  # same length as x
plt.plot(x, y, 'o-'); plt.show()


### 7) Understanding error messages and applying corrections — Practice & Fixes


In [ ]:

# Try/except with detailed traceback and fallback when reading a problematic CSV
import traceback
try:
    df2 = pd.read_csv('possibly_corrupt.csv', parse_dates=['date'])
except Exception:
    print('Failed to read CSV:', traceback.format_exc())
    df2 = pd.read_csv('possibly_corrupt.csv', engine='python', on_bad_lines='warn')
    df2['date'] = pd.to_datetime(df2['date'], errors='coerce')

# Assert expected schema
def assert_schema(df_in, expected):
    """expected: dict like {'date': 'datetime64[ns]', 'revenue': 'float64'}"""
    actual = df_in.dtypes.astype(str).to_dict()
    for col, dtype in expected.items():
        if col not in actual:
            raise AssertionError(f'Missing column: {col}')
        if actual[col] != dtype:
            raise AssertionError(f"Column '{col}' has dtype {actual[col]} but expected {dtype}")
    return True

# Example assertion (adjust expected types to your data)
# assert_schema(df, {'date': 'datetime64[ns]', 'revenue': 'float64', 'category': 'category'})

# Check sorted index and duplicates
daily2 = df.groupby('date')['revenue'].sum()
if not daily2.index.is_monotonic_increasing:
    print('Warning: Dates are not sorted. Sorting now.')
    daily2 = daily2.sort_index()
if daily2.index.duplicated().any():
    print(f'Warning: Duplicate dates found: {daily2.index.duplicated().sum()} duplicates')


---
### Next steps
- Replace file names with your actual CSVs.- Run cells top to bottom.- If you want me to tailor this to your exact schema (column names), share a sample of your CSV.
